In [ ]:
import quandl
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mdates
import numpy as np
quandl.ApiConfig.api_key = 's_YHBWCvr5YJr3-2ifzU' # key 

In [ ]:
data = quandl.get('EURONEXT/', start_date='2018-01-02', end_date='2018-12-04')

In [ ]:
data.Close.plot()
plt.show()

In [ ]:
data_analysis = data

# basic analysis

In [ ]:
data_analysis['100ma'] = data_analysis['Close'].rolling(window=100,min_periods=0).mean()
# this is used to compute the average for pass 100 price (including itself)

ax1 = plt.subplot2grid((6,1),(0,0),rowspan=5,colspan=1)
ax2 = plt.subplot2grid((6,1),(5,0),rowspan=5,colspan=1,sharex=ax1)
ax1.plot(data.index,data_analysis['Close'])
ax1.plot(data.index,data_analysis['100ma'])
ax2.plot(data.index,data_analysis['Total Trade Quantity'])
plt.show()

In [ ]:
data_ohlc = data_analysis['Close'].resample('10D').ohlc()
data_volume = data_analysis['Total Trade Quantity'].resample('10D').sum()
data_ohlc.reset_index(inplace=True) # the index should be 0 1 2 ....
data_ohlc['Date'] = data_ohlc['Date'].map(mdates.date2num) # transfer datetime to timestamp

ax1 = plt.subplot2grid((6,1),(0,0),rowspan=5,colspan=1)
ax2 = plt.subplot2grid((6,1),(5,0),rowspan=5,colspan=1,sharex=ax1)
ax1.xaxis_date()
candlestick_ohlc(ax1,data_ohlc.values,width=2,colorup='g')
ax2.fill_between(data_volume.index.map(mdates.date2num),data_volume.values,0)
plt.show() ## candlestick graph

## calculation of Relative Strength index(RSI)

In [ ]:
TechIndicator = data.copy()

In [ ]:
# Relative Strength Index
# Avg(PriceUp)/(Avg(PriceUP)+Avg(PriceDown)*100
# Where: PriceUp(t)=1*(Price(t)-Price(t-1)){Price(t)- Price(t-1)>0};
#        PriceDown(t)=-1*(Price(t)-Price(t-1)){Price(t)- Price(t-1)<0};

def rsi(values):
    up = values[values>0].mean()
    down = -1*values[values<0].mean()
    return 100 * up / (up + down)

In [ ]:
# Add Momentum_1D column for all 15 stocks.
# Momentum_1D = P(t) - P(t-1)
TechIndicator['Momentum_1D'] = (TechIndicator['Close']-TechIndicator['Close'].shift(1)).fillna(0)
TechIndicator['RSI_14D'] = TechIndicator['Momentum_1D'].rolling(center=False, window=14).apply(rsi).fillna(0)
TechIndicator.tail(5)

## Calculation of Bollinger Bands

In [ ]:
def bbands(price, length=30, numsd=2):
    """ returns average, upper band, and lower band"""
    #ave = pd.stats.moments.rolling_mean(price,length)
    ave = price.rolling(window = length, center = False).mean()
    #sd = pd.stats.moments.rolling_std(price,length)
    sd = price.rolling(window = length, center = False).std()
    upband = ave + (sd*numsd)
    dnband = ave - (sd*numsd)
    return np.round(ave,3), np.round(upband,3), np.round(dnband,3)

In [ ]:
TechIndicator['BB_Middle_Band'], TechIndicator['BB_Upper_Band'], TechIndicator['BB_Lower_Band'] = bbands(TechIndicator['Close'], length=20, numsd=1)
TechIndicator['BB_Middle_Band'] = TechIndicator['BB_Middle_Band'].fillna(0)
TechIndicator['BB_Upper_Band'] = TechIndicator['BB_Upper_Band'].fillna(0)
TechIndicator['BB_Lower_Band'] = TechIndicator['BB_Lower_Band'].fillna(0)
TechIndicator.tail()

## Calculation of Aroon Oscillator

In [ ]:
def aroon(df, tf=25):
    aroonup = []
    aroondown = []
    x = tf
    while x< len(df['High']):
        aroon_up = ((df['High'][x-tf:x].tolist().index(max(df['High'][x-tf:x])))/float(tf))*100
        aroon_down = ((df['Low'][x-tf:x].tolist().index(min(df['Low'][x-tf:x])))/float(tf))*100
        aroonup.append(aroon_up)
        aroondown.append(aroon_down)
        x+=1
    return aroonup, aroondown

In [ ]:
listofzeros = [0] * 25
up, down = aroon(TechIndicator)
aroon_list = [x - y for x, y in zip(up,down)]
if len(aroon_list)==0:
    aroon_list = [0] * TechIndicator.shape[0]
    TechIndicator['Aroon_Oscillator'] = aroon_list
else:
    TechIndicator['Aroon_Oscillator'] = listofzeros+aroon_list

## Calculation of Price Volume Trend
PVT = [((CurrentClose - PreviousClose) / PreviousClose) x Volume] + PreviousPVT

In [ ]:
TechIndicator["PVT"] = (TechIndicator['Momentum_1D']/ TechIndicator['Close'].shift(1))*TechIndicator['Total Trade Quantity']
TechIndicator["PVT"] = TechIndicator["PVT"]-TechIndicator["PVT"].shift(1)
TechIndicator["PVT"] = TechIndicator["PVT"].fillna(0)

## Calculation of Acceleration Bands

# Input data from yahoo 

In [3]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
yf.pdr_override()


Since we don't have the tickers names of companies in LSE(London Stock Exchange), we need to input all tickers names at first 

In [4]:
data = pd.read_csv('LSE.txt')

In [5]:
def splitTicker(row):
    splitname = row['Symbol\tDescription'].split('\t')
    return splitname[0]
def splitCompany(row):
    splitname = row['Symbol\tDescription'].split('\t')
    return splitname[1]
def lowerName(row):
    new_name =  row['company_name'].lower()
    return new_name
def transferDatetime(row):
   
    time = datetime.strptime(str(row['addmission_time']), '%d/%m/%Y')
    return time

def filterAddmissionTime(data,past_time):
    return data[data['addmission_time']<=past_time]

In [6]:
data['ticker']= data.apply(splitTicker,axis=1)
data['company_name'] = data.apply(splitCompany,axis=1)
data['company_name'] = data.apply(lowerName,axis=1)
data['company_name'].head()

0    statoilhydro asa 6.125% notes 27/11/28 gbp(var)
1                                 baywa ag npv(vink)
2                       crcam nord de france eur3.05
3                                 carlsberg `a`dkk20
4                                 carlsberg `b`dkk20
Name: company_name, dtype: object

In [7]:
company = pd.read_csv('company.csv')
company = company[['Unnamed: 1','Unnamed: 2']][5:-2]
company.columns=['addmission_time','company_name']
company['company_name'] = company.apply(lowerName,axis=1)
company.dropna(inplace=True)
company['addmission_time'] = company.apply(transferDatetime,axis=1)

In [8]:
cleaned_data = company.merge(data,on=['company_name']).drop(columns='Symbol\tDescription')
cleaned_data.head()

,addmission_time,company_name,ticker
0,2006-08-02 00:00:00,1pm plc,OPM.L
1,2009-02-02 00:00:00,1spatial plc,SPA.L
2,1994-07-18 00:00:00,3i group plc,III.L
3,2014-02-18 00:00:00,4d pharma plc,DDDD.L
4,1953-03-13 00:00:00,4imprint group plc,FOUR.L


In [9]:
past_time = datetime.now()-timedelta(days=3650)

filtered_date = filterAddmissionTime(cleaned_data,past_time)

In [10]:
tickers_list =  list(filtered_date['ticker'])

In [ ]:
pdr.get_data_yahoo(tickers=tickers_list,start=past_time,end=datetime.now())

In [ ]:
tickers_list